# Introduction 
`V1.0.1 - 2021-10-18`
### Who am I
Just a fellow Kaggle learner. I was creating this Notebook as practice and thought it could be useful to some others 
### Who is this for
This Notebook is for people that learn from examples. Forget the boring lectures and follow along for some fun/instructive time :)
### What can I learn here
You learn all the basics needed to create a rudimentary LSTM Encoder-Decoder Network. I go over a multitude of steps with explanations. Hopefully with these building blocks,
you can go ahead and build much more complex models.

### Thins to remember
+ Please Upvote/Like the Notebook so other people can learn from it
+ Feel free to give any recommendations/changes. 
+ I will be continuously updating the notebook. Look forward to many more upcoming changes in the future.

# Imports
First let us start by importing the relevant libraries that we need.

In [ ]:
# Computational imports
import numpy as np   # Library for n-dimensional arrays
import pandas as pd  # Library for dataframes (structured data)

# Helper imports
import os 
import warnings
import pandas_datareader as web
import datetime as dt

# ML/DL imports
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Plotting imports
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
from numpy.random import seed
seed(1)

# Helper Functions
These functions will help us prepare our data to be fed into the model

In [ ]:
# split a univariate sequence into samples
def split_sequence(data, days_past, days_future):
    X, y = list(), list()
    
    for i in range(len(data)):        
        # find the end of this pattern
        end_ix = i + days_past
        out_end_ix = end_ix + days_future
        
        # check if we are beyond the sequence
        if out_end_ix > len(data):
            break
            
        # gather input and output parts of the pattern
        seq_x, seq_y = data[i:end_ix], data[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

# Reading and Preparing the Data
Let's start by preparing our data. We will store it in a dataframe. We are only interested in the sales number.

In [ ]:
file_name = '../input/demand-forecasting-kernels-only/train.csv'
dataset = pd.read_csv(file_name, parse_dates=['date'])

dataset.drop(labels=['store', 'item'], axis=1, inplace = True)

In [ ]:
dataset.head()

In [ ]:
dataset.describe()

# Time period of the dataset
Let's see what is the start and end of the dataset

In [ ]:
print('Min date from train set: %s' % dataset.date.min().date())
print('Max date from train set: %s' % dataset.date.max().date())

# Checking for NaN (missing values) in the dataframe
There is multiple ways of doing this, I have chosen do it with the following method:

In [ ]:
dataset.sales.isnull().sum()

This dataset is pretty easy, no missing data... less work for us :)

# Splitting the data set in training and test sets
This part is important since we will be using the training set to train the model and the test set to check if the model does well on unseen inputs.

We are going to use only a small subset of the data due to training time. In competitons, use as much data that there is avaialable to botrain a stronger model that is good at generalizing. 

In [ ]:
training_data = dataset[(dataset['date'] <= '2016')]
test_data = dataset[(dataset['date'] >= '2017')]

training_data.sort_values('date', inplace=True)
training_data = training_data.groupby('date')
training_data = training_data.agg({'sales':'mean'})
training_data.colums = ['sales']

test_data.sort_values('date', inplace=True)
test_data = test_data.groupby('date')
test_data = test_data.agg({'sales':'mean'})
test_data.colums = ['sales']

print(len(training_data))
print(len(test_data))

In [ ]:
training_data.head()

# Visualizing and Plotting our Data
It is very important to plot and visualize the data to get a good grasp of what is happening. For this, I have decided to use plotly, but you can use any other visualizaiton library such as seaborn and matplotlib. We are using plotly graph objects since we will need to create subplots later.

In [ ]:
fig  = make_subplots()
fig.add_trace(go.Scatter(x=training_data.index,y=training_data.sales,name="Zoom"))
fig.update_layout(autosize=True,width=900,height=500,title_text="Sales")
fig.update_xaxes(title_text="year")
fig.update_yaxes(title_text="prices")
fig.show()

Now to plot the training and test data on the same plot. Notice the lag between the training data and test data. This is done on purpose to ensure that the test data isn't too strongly correlated to the training data.

In [ ]:
fig  = make_subplots()
fig.add_trace(go.Scatter(x=training_data.index,y=training_data.sales,name="Train"))
fig.add_trace(go.Scatter(x=test_data.index,y=test_data.sales,name="Test"))
fig.update_layout(autosize=False,width=900,height=500,title_text="Sales")
fig.update_xaxes(title_text="day")
fig.update_yaxes(title_text="prices")
fig.show()

# Scaling the training data
Here we will use MinMaxScaler from the Sci-kit learn library to standardize and scale our input data between 0-1.

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_tain_data = scaler.fit_transform(training_data.values.reshape(-1,1))
scaled_tain_data.shape

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_test_data = scaler.fit_transform(test_data.values.reshape(-1,1))
scaled_test_data.shape

# Transforming the data into a time-series appropriate problem
We will now have to rpepare the data in a form that is appropriate to be fed into a RNN/LSTM network. The key is to seperate the data into past sequence (Day 0-29) that is going to be used to predict the future sequence (Day 30-34) for example.

In [ ]:
days_past = 30   # use past 30 time steps to predict
days_future = 5  # predict 5 time steps in the future

x_train, y_train = split_sequence(scaled_tain_data, days_past, days_future)
x_test, y_test = split_sequence(scaled_test_data, days_past, days_future)

In [ ]:
print(x_train.shape)
print(x_test.shape)

The LSTM model expects the input to have shape: (Samples, Timesteps, Number of feautures). In our case, we have an univariate RNN problem, therefore Number of feautures = 1

In [ ]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))

x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], 1))

In [ ]:
print(x_train.shape)
print(x_test.shape)

# Creating the LSTM Network
We are going to be creating a three layer LSTM Network with a dense layer at the end. We are using dropout as a regularisation method to combat overffiting.

In [ ]:
def encoder_decoder_model():
    
    # Use Keras sequential model
    model = Sequential()
    
    # Encoder LSTM layer with Dropout regularisation; Set return_sequences to False since we are feeding last output to decoder layer
    model.add(LSTM(100, activation='relu',input_shape = (days_past,1)))
    model.add(Dropout(0.2))
    
    # The fixed-length output of the encoder is repeated, once for each required time step in the output sequence with the RepeatVector wrapper
    model.add(RepeatVector(days_future))
    
    # Decoder LSTM layer with Dropout regularisation; Set return_sequences to True to feed each output time step to a Dense layer
    model.add(LSTM(100, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    
    # Same dense layer is repeated for each output timestep with the TimeDistributed wrapper
    model.add(TimeDistributed(Dense(units=1, activation = "linear")))
    
    return model

In [ ]:
model = encoder_decoder_model()
model.summary()

Now we set our compiler and our optimatization mechanism. We will be using the Adam optimazation method since it is widely used and performs much better than regular gradient descent.

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['accuracy'])

# Training/Fitting time
We can finally train our model with our training data. Let's see how it does.

In [ ]:
checkpointer = ModelCheckpoint(filepath = './checkpoint.hdf5', verbose = 0, save_best_only = True)
his=model.fit(x_train,y_train,epochs=50,callbacks=[checkpointer], verbose=0)

Let us now visualize the loss and accuracy to see how the training went.

In [ ]:
plt.plot(his.history['loss'])
plt.plot(his.history['accuracy'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss','accuracy'])
plt.show()

We see that the loss decreaes and is lower than 0.2 MSE. This shows that our model is indeed doing well on the training data.

# Testing Time
Now that we have validated that the model does pretty well on our training data, we can move to some more serious stuff... TEST DATA
Why is this important you might ask? Well if you score plenty of goals in your practices (good job I guess), but none on the real game... there must be something wrong.

That is why we need test data to confirm that our model does well on unseen data.

In [ ]:
# Predicting the prices
predicted_sales = model.predict(x_test)

# Test Visualization
Let's conclude this notebook with some good ole data visualization. We'll start by flattening the 2D array into an 1D array. This is done for ease of plotting.

In [ ]:
# We upse numpy squeeze to transform (samples, timesteps, number of features) into (samples, timesteps), since that is what inverse_transform expects
y_test = scaler.inverse_transform(np.squeeze(y_test))
predicted_sales = scaler.inverse_transform(np.squeeze(predicted_sales))


# We flatten the 2 dimensional array so we can plot it with matplotlib
y_test = y_test.flatten()
predicted_sales = predicted_sales.flatten()

In [ ]:
plt.plot(y_test, color='black', label=f"Actual Sales")
plt.plot(predicted_sales, color= 'green', label="Predicted Sales")
plt.title("Sales vs Predicted Sales")
plt.xlabel("Days in test period")
plt.ylabel("Price")
plt.legend()
plt.show()

# Final Remarks
I hope this notebook helped you a bit. If so, please upvote the notebook :)!